In [7]:
import os
import pandas as pd
from pandas import DataFrame

def merge_data(station_paths:list[str],info:str)->DataFrame:
    '''
    傳出所有資料
    '''
   # print(station_paths)
    #print("========================")
    #print(info)
    info_df = pd.read_csv(info)
    merge_datas = []
    for sta_path in station_paths:
        station_df = pd.read_csv(sta_path)     
        merge_df = pd.merge(station_df,info_df, how='left',left_on='staCode',right_on='stationCode')
        merge_datas.append(merge_df)
    return pd.concat(merge_datas)
current_folder = os.getcwd()     #取得現在的目錄
stations_folder = os.path.join(current_folder,'stations')
csv_abs_files = []
for filename in os.listdir(stations_folder):
    if '每日各站' in filename:
        os.path.join(stations_folder,filename)
        csv_abs_files.append(os.path.join(stations_folder,filename))

info_path = os.path.join(stations_folder,'台鐵車站資訊.csv')

df = merge_data(csv_abs_files,info_path)
df1 = df[['trnOpDate','gateInComingCnt','gateOutGoingCnt','name','stationAddrTw','haveBike']]


In [ ]:
df1.columns = ['日期','進站人數','出站人數','站名','地址','YOUBIKE']
df1

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 0 to 86551
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   日期       406761 non-null  int64 
 1   進站人數     406761 non-null  int64 
 2   出站人數     406761 non-null  int64 
 3   站名       406761 non-null  object
 4   地址       406761 non-null  object
 5   YOUBIKE  406761 non-null  object
dtypes: int64(3), object(3)
memory usage: 21.7+ MB


In [13]:
df1['日期'] = pd.to_datetime(df1['日期'].astype(str))#先把serial轉成字串再轉成datetime


/tmp/ipykernel_28843/2238973697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['日期'] = pd.to_datetime(df1['日期'].astype(str))#先把serial轉成字串再轉成datetime


In [ ]:
df1

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 0 to 86551
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   日期       406761 non-null  datetime64[ns]
 1   進站人數     406761 non-null  int64         
 2   出站人數     406761 non-null  int64         
 3   站名       406761 non-null  object        
 4   地址       406761 non-null  object        
 5   YOUBIKE  406761 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 21.7+ MB


In [17]:
df1['YOUBIKE']=df1['YOUBIKE'].map({'Y':1,'N':0})

/tmp/ipykernel_28843/768175867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['YOUBIKE']=df1['YOUBIKE'].map({'Y':1,'N':0})


In [ ]:
df1

In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 0 to 86551
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   日期       406761 non-null  datetime64[ns]
 1   進站人數     406761 non-null  int64         
 2   出站人數     406761 non-null  int64         
 3   站名       406761 non-null  object        
 4   地址       406761 non-null  object        
 5   YOUBIKE  406761 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 21.7+ MB


In [ ]:
df2 = df1.sort_values(by='日期')#依日期排序
df3 = df2.set_index('日期')     #將索引值改成日期
df3

In [23]:
def first_third(val:str)->str:
    return val[:3]  #字串是序列資料,所以可以用:來取前3個
df3['地址'].map(first_third)

日期
2019-04-23    臺南市
2019-04-23    彰化縣
2019-04-23    南投縣
2019-04-23    南投縣
2019-04-23    南投縣
             ... 
2023-12-31    彰化縣
2023-12-31    彰化縣
2023-12-31    彰化縣
2023-12-31    臺中市
2023-12-31    基隆市
Name: 地址, Length: 406761, dtype: object

In [ ]:
df3['縣市'] = df3['地址'].map(lambda v:v[:3])
df3

In [27]:
df3.index.day       #如果索引是timestamp,就可以用.day的這種工具來抓出日

Index([23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
       ...
       31, 31, 31, 31, 31, 31, 31, 31, 31, 31],
      dtype='int32', name='日期', length=406761)

In [31]:
df3['年度'] = df3.index.strftime('%Y')
df3['年/月'] = df3.index.strftime('%Y/%m')

In [ ]:
df4 = df3.reindex(columns=['縣市','年度','年/月','站名','進站人數','出站人數','YOUBIKE'])
df4

In [37]:
df4.to_csv('全台火車進出站資料.csv')